In [ ]:
# Create separate DataFrames for each branch
branch_a_df = df1[df1['branch'] == 'A']
branch_b_df = df1[df1['branch'] == 'B']
branch_c_df = df1[df1['branch'] == 'C']

# Aggregate data for branch A
branch_a_agg = branch_a_df.groupby('date').agg({
    'unit_price': 'mean',
    'quantity': 'sum',
    'rating': 'mean',
    'day': 'first',
    'month': 'first',
    'year': 'first',
    'gross_income': 'sum'
}).reset_index()

# Aggregate data for branch B
branch_b_agg = branch_b_df.groupby('date').agg({
    'unit_price': 'mean',
    'quantity': 'sum',
    'rating': 'mean',
    'day': 'first',
    'month': 'first',
    'year': 'first',
    'gross_income': 'sum'
}).reset_index()

# Aggregate data for branch C
branch_c_agg = branch_c_df.groupby('date').agg({
    'unit_price': 'mean',
    'quantity': 'sum',
    'rating': 'mean',
    'day': 'first',
    'month': 'first',
    'year': 'first',
    'gross_income': 'sum'
}).reset_index()

# Add branch identifier
branch_a_agg['branch'] = 'A'
branch_b_agg['branch'] = 'B'
branch_c_agg['branch'] = 'C'

# Concatenate the DataFrames
combined_df = pd.concat([branch_a_agg, branch_b_agg, branch_c_agg], ignore_index=True)


# Convert the 'date' column to datetime format
combined_df['date'] = pd.to_datetime(combined_df['date'])

# Sort the dataset by the 'date' column
combined_df = combined_df.sort_values(by='date').reset_index(drop=True)
combined_df 


# Encoding Categorical Variables
# One-hot encode the 'branch' column
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first', sparse=False)
branch_encoded = encoder.fit_transform(combined_df[['branch']])

# Create a DataFrame with the one-hot encoded branch data
branch_encoded_df = pd.DataFrame(branch_encoded, columns=encoder.get_feature_names_out(['branch']))

# Concatenate the one-hot encoded columns to the original dataframe
combined_df_encoded = pd.concat([combined_df.drop(columns=['branch']), branch_encoded_df], axis=1)
combined_df_encoded


#### Shift Lag and Rolling Lag Moving Average:
Shift quantity and unit_price by 1 period to create lagged versions.
# Create lagged versions of 'quantity' and 'unit_price'
combined_df_encoded['quantity_lag1'] = combined_df_encoded['quantity'].shift(1)
combined_df_encoded['unit_price_lag1'] = combined_df_encoded['unit_price'].shift(1)

# Create rolling averages of 'quantity' and 'unit_price' with a window of 7
combined_df_encoded['quantity_roll7'] = combined_df_encoded['quantity'].rolling(window=7).mean()
combined_df_encoded['unit_price_roll7'] = combined_df_encoded['unit_price'].rolling(window=7).mean()
combined_df_encoded


# Step 2: Transform 'day' column into sine and cosine components
combined_df_encoded['day_sin'] = np.sin(2 * np.pi * combined_df_encoded['day'] / 31)
combined_df_encoded['day_cos'] = np.cos(2 * np.pi * combined_df_encoded['day'] / 31)
combined_df_encoded


# Calculate the number of rows before dropping NaN values
rows_before = combined_df_encoded.shape[0]

# Drop rows with NaN values
combined_df_encoded_dropped = combined_df_encoded.dropna().reset_index(drop=True)

# Calculate the number of rows after dropping NaN values
rows_after = combined_df_encoded_dropped.shape[0]

# Number of rows lost
rows_lost = rows_before - rows_after

rows_before, rows_after, rows_lost


# Drop rows with NaN values resulting from the lag and rolling operations
combined_df_encoded = combined_df_encoded.dropna().reset_index(drop=True)
combined_df_encoded